In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy.optimize import minimize
import random

In [ ]:
rdf = pd.read_csv('winequality-red.csv', sep=';')


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


Here we will do a 75%-15%-10% test split for train-validation-test respectively

In [14]:
rdf_train, rdf_val = train_test_split(rdf, test_size=0.25, random_state=50)
rdf_val, rdf_test = train_test_split(rdf_val, test_size=0.4, random_state=50)

In [ ]:
X_train = rdf_train.drop('quality', axis=1)
y_train = rdf_train['quality']
X_test = rdf_test.drop('quality', axis=1)
y_test = rdf_test['quality']

Standardization:


In [ ]:
#preprocessing

#standardization
X_train_values = X_train.values
X_test_values = X_test.values

X_train_mean = np.mean(X_train_values, axis=0)
X_train_std = np.std(X_train_values, axis=0)

def standardize(values, mean, std):
    return (values - mean) / std

X_train_standardized = standardize(X_train_values, X_train_mean, X_train_std)
X_test_standardized = standardize(X_test_values, X_train_mean, X_train_std)

#normilzation
X_train_min = np.min(X_train_values, axis=0)
X_train_max = np.max(X_train_values, axis=0)

def normalize(values,mins,maxs):
    return (values - mins) / (maxs - mins)

X_train_normalized = normalize(X_train_values, X_train_min, X_train_max)
X_test_normalized = normalize(X_test_values, X_train_min, X_train_max)

In [ ]:
#Ridge Regression

def m_ridge(X,t,lam):
    X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
    I = np.eye(X.shape[1])
    #I[0,0] = 0
    return np.linagl.inv((lam*I) + (X.T @ X)) @ (X.T @ t) #minimizer of the ridge regression

w_m_ridge_trs = m_ridge(X_train_standardized, y_train, 0.1) #minimizer with standardized train set
w_m_ridge_trn = m_ridge(X_train_normalized, y_train, 0.1) #minimizer with normalized train set
w_m_ridge_tss = m_ridge(X_test_standardized, y_train, 0.1) #minimizer with standardized test set
w_m_ridge_tsn = m_ridge(X_test_normalized, y_train, 0.1) #minimizer with normalized test set

def ridge_regr(w,t,X,lam):
    X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
    return ((1/2)*np.sum((t-X@w)**2))+((lam/2)*(w.T@w)) #this is the ridge regression formula #try np.sum(w**2) 

#Lasso Regression

def lasso_regr(w,t,X,lam):
    X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
    return ((1/2)*np.sum((t-X@w)**2))+(lam*np.sum(np.abs(w))) #this is the lasso regression formula

def m_lasso(X,t,lam):
    w_init = np.zeros(X.shape[1])
    result = minimize(lasso_regr,w_init,arg=(X,t,lam),method='SLSQP')
    return result.x #minimizer of the lasso regression using sequential least squares programming 

w_m_lasso_trs = m_lasso(X_train_standardized, y_train, 0.1)
w_m_lasso_trn = m_lasso(X_train_normalized, y_train, 0.1) 
w_m_lasso_tss = m_lasso(X_test_standardized, y_train, 0.1)
w_m_lasso_tsn = m_lasso(X_test_normalized, y_train, 0.1) 

# Regular Linear Regression

def m_regression(X,t):
    X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
    return np.linalg.pinv(X)@t #minimizer of the sum of squares error

w_m_regr_trs = m_regression(X_train_standardized, y_train, 0.1)
w_m_regr_trn = m_regression(X_train_normalized, y_train, 0.1) 
w_m_regr_tss = m_regression(X_test_standardized, y_train, 0.1)
w_m_regr_tsn = m_regression(X_test_normalized, y_train, 0.1) 

def lin_regr(w,t,X): 
    X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
    return (1/2)*np.sum((t - X@w)**2) #sum of squares error for regression



In [ ]:
def rmse(regr,N): #Root mean square error
    return np.sqrt((2*regr)/N)